# Notebook for retrieving final destination phishing domains from urlscan.io 

## Requirements
* Python Libraries
  * Pandas
  * Requests
* urlscan.io API Key

In [ ]:
# Libs
from requests import get
from getpass import getpass
from urllib.parse import quote
from json import loads
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

## Get urlscan.io data

### REGEXs
We have 3 REGEX queries that will find all the relevant final destination (FD) phishing domains with high accuracy. The first 2 queries are not combined due to limitations in how many results you can obtain at one time from the urlscan API. 

#### REGEX 1: All FD URLs abusing reCAPTCHA with `-`
**Base REGEX**

```
^(http)(s)?:\/\/([a-z0-9]+\.)?[a-z0-9]{1,}\-[a-z0-9]{1,}\.[a-z]+\/(main|jump)\/$
```

**urlscan.io Query**

```
filename:"/recaptcha/api.js" AND page.url.keyword:/(http)(s)?:\/\/([a-z0-9]+\.)?[a-z0-9]{1,}\-[a-z0-9]{1,}\.[a-z]+\/(main|jump)\//
```


#### REGEX 2: All FD URLs abusing reCAPTCHA without `-`
**Base REGEX**

```
^(http)(s)?:\/\/([a-z0-9]+\.)?([a-z]|[0-9])+\.[a-z]+\/(main|jump)\/$
```

**urlscan.io Query**
```
filename:"/recaptcha/api.js" AND page.url.keyword:/(http)(s)?:\/\/([a-z0-9]+\.)?([a-z]|[0-9])+\.[a-z]+\/(main|jump)\//
```


#### REGEX 3: All FD URLs Serving ExRobotos Phishing Kit
**Base REGEX**

```
^(http)(s)?:\/\/([a-z0-9]+\.)?([a-z0-9\-])+\.([a-z])+\/(.*)?\.well-known\/((login\.php\?ss=2(&)?.*)|(.*\/authorize_client_id\:.*))+$
```

**urlscan.io Query**
```
page.url.keyword:/(http)(s)?:\/\/([a-z0-9]+\.)?([a-z0-9\-])+\.([a-z])+\/(.*)?\.well-known\/((login\.php\?ss=2(&)?.*)|(.*\/authorize_client_id\:.*))+/
```


### Getting Data
Using python requests can sometimes be slow, so if you are using curl, you can also load results from a file.
The first code block uses Python's requests library while the next code block loads from a file.

#### Python Requests

**Enter API Key**

In [ ]:
api_key = getpass()

In [ ]:
# Set max results here!
max_results = 10000
base_url = f'https://urlscan.io/api/v1/search/?size={max_results}&q='
regex_one_query = quote(
    'filename:"/recaptcha/api.js" AND page.url.keyword:/(http)(s)?:\/\/([a-z0-9]+\.)?[a-z0-9]{1,}\-[a-z0-9]{1,}\.(shop|online|xyz|club|com|top|si|org|net|live)\/(main|jump)\//'
)
regex_two_query = quote(
    'filename:"/recaptcha/api.js" AND page.url.keyword:/(http)(s)?:\/\/([a-z0-9]+\.)?([a-z]|[0-9])+\.[a-z]+\/(main|jump)\//'
)
regex_three_query = quote(
    'page.url.keyword:/(http)(s)?:\/\/([a-z0-9]+\.)?([a-z0-9\-])+\.([a-z])+\/(.*)?\.well-known\/((login\.php\?ss=2(&)?.*)|(.*\/authorize_client_id\:.*))+/'
)
headers = {'api-key': api_key}

In [ ]:
regex_one_results = get(f'{base_url}{regex_one_query}', headers=headers).json()

In [ ]:
regex_two_results = get(f'{base_url}{regex_two_query}', headers=headers).json()

In [ ]:
regex_three_results = get(f'{base_url}{regex_three_query}', headers=headers).json()

#### Parse results from json file

In [ ]:
# Parse results from json file
regex_one_file = ''
regex_two_file = ''
regex_three_file = ''

In [ ]:
with open(regex_one_file, 'r') as file:
    regex_one_results = loads(file.read())

In [ ]:
with open(regex_two_file, 'r') as file:
    regex_two_results = loads(file.read())

In [ ]:
with open(regex_three_file, 'r') as file:
    regex_three_results = loads(file.read())

In [ ]:
print(f'REGEX one results: {regex_one_results["total"]}')
print(f'REGEX two results: {regex_two_results["total"]}')
print(f'REGEX three results: {regex_three_results["total"]}')

# Combine data
recaptcha_results = regex_one_results['results'] + regex_two_results['results']
exrobotos_results = regex_three_results['results']
total_recaptcha = len(recaptcha_results)
total_exrobotos = len(exrobotos_results)
print(f'Total reCAPTCHA results: {total_recaptcha}')
print(f'Total ExRobotos results: {total_exrobotos}')

## Output Final Destination Phishing Domains

**Enter output files**

In [ ]:
output_recaptcha_file = 'phish_domains_abusing_recaptcha.txt'
output_exrobotos_file = 'phish_domains_exrobotos.txt'

In [ ]:
# Dedup domains
recaptcha_domains = set()
for i in recaptcha_results:
    recaptcha_domains.add(i['page']['domain'])

print(f'Total phishing domains abusing reCAPTCHA: {len(recaptcha_domains)}')

# Write to file
with open(output_recaptcha_file, 'w+') as file:
    for domain in recaptcha_domains:
        file.write(f"{domain}\n")

# Dedup domains
exrobotos_domains = set()
for i in exrobotos_results:
    exrobotos_domains.add(i['page']['domain'])
print(f'Total phishing domains using ExRobotos Phishing Kit: {len(exrobotos_domains)}')

# Write to file
with open(output_exrobotos_file, 'w+') as file:
    for domain in exrobotos_domains:
        file.write(f"{domain}\n")

**Optional: output phish domains with datetime**

In [ ]:
output_recaptcha_file = 'phish_domains_abusing_recaptcha_with_datetime.csv'
output_exrobotos_file = 'phish_domains_exrobotos_with_datetime.csv'

In [ ]:
tasks_and_page_urls = []
for i in recaptcha_results:
    tasks_and_page_urls.append([i['task']['time'], i['page']['domain']])
df = pd.DataFrame(tasks_and_page_urls, columns=['Task Time', 'Domain'])

print(f'Total results: {len(tasks_and_page_urls)}')

# Write to file
with open(output_recaptcha_file, 'w+') as file:
    file.write(df.to_csv(index=False))

    
tasks_and_page_urls = []
for i in exrobotos_results:
    tasks_and_page_urls.append([i['task']['time'], i['page']['domain']])
df = pd.DataFrame(tasks_and_page_urls, columns=['Task Time', 'Domain'])

print(f'Total results: {len(tasks_and_page_urls)}')

# Write to file
with open(output_exrobotos_file, 'w+') as file:
    file.write(df.to_csv(index=False))